In [1]:
import tkinter as tk
from tkinter import filedialog, messagebox, font
import time

# Brute Force Search Function
def brute_force_search(text, pattern, case_sensitive=False, whole_word=False):
    if not case_sensitive:
        text = text.lower()
        pattern = pattern.lower()
    
    results = []
    lines = text.splitlines()
    for row, line in enumerate(lines):
        if not case_sensitive:
            line = line.lower()
        for col in range(len(line) - len(pattern) + 1):
            if line[col:col+len(pattern)] == pattern:
                if whole_word:
                    if (col == 0 or not line[col-1].isalnum()) and (col + len(pattern) == len(line) or not line[col+len(pattern)].isalnum()):
                        results.append((row + 1, col + 1))
                else:
                    results.append((row + 1, col + 1))
    return results

# Knuth-Morris-Pratt (KMP) Search Function
def kmp_search(text, pattern, case_sensitive=False, whole_word=False):
    if not case_sensitive:
        text = text.lower()
        pattern = pattern.lower()

    lps = [0] * len(pattern)
    j = 0
    for i in range(1, len(pattern)):
        if pattern[i] == pattern[j]:
            j += 1
            lps[i] = j
        else:
            if j > 0:
                j = lps[j - 1]
                i -= 1

    results = []
    i = j = 0
    lines = text.splitlines()
    row, col = 1, 0
    while row <= len(lines):
        line = lines[row - 1]
        if i < len(line) and pattern[j] == line[i]:
            i += 1
            j += 1
        if j == len(pattern):
            if whole_word:
                if (i - j == 0 or not line[i - j - 1].isalnum()) and (i == len(line) or not line[i].isalnum()):
                    results.append((row, i - j + 1))
            else:
                results.append((row, i - j + 1))
            j = lps[j - 1]
        elif i < len(line) and pattern[j] != line[i]:
            if j > 0:
                j = lps[j - 1]
            else:
                i += 1
        if i >= len(line):
            row += 1
            i = 0
            j = 0
    return results

# Main GUI Application
class WordSearchApp:
    def __init__(self, root):
        self.root = root
        self.root.title("Word Search Application")
        self.root.configure(bg="#282c34")

        # Custom fonts
        title_font = font.Font(family="Helvetica", size=14, weight="bold")
        label_font = font.Font(family="Helvetica", size=10)
        button_font = font.Font(family="Helvetica", size=10, weight="bold")
        
        self.file_contents = {}  # Dictionary to hold loaded files and their contents

        # Title Label
        self.title_label = tk.Label(root, text="Word Search Application", font=title_font, fg="#61afef", bg="#282c34")
        self.title_label.grid(row=0, column=0, columnspan=2, pady=(10, 20))

        # Load Files Button
        self.load_files_button = tk.Button(root, text="Load Files", font=button_font, command=self.load_files, fg="#ffffff", bg="#61afef")
        self.load_files_button.grid(row=1, column=0, columnspan=2, padx=10, pady=5)

        # Search Term Entry
        self.search_term_label = tk.Label(root, text="Enter search term:", font=label_font, fg="#abb2bf", bg="#282c34")
        self.search_term_label.grid(row=2, column=0, padx=10, pady=5, sticky="e")
        self.search_term_entry = tk.Entry(root, width=30, font=label_font)
        self.search_term_entry.grid(row=2, column=1, padx=10, pady=5)

        # Options Checkbuttons
        self.whole_word_var = tk.BooleanVar()
        self.whole_word_check = tk.Checkbutton(root, text="Whole Word Match", font=label_font, variable=self.whole_word_var, fg="#d19a66", bg="#282c34", selectcolor="#282c34")
        self.whole_word_check.grid(row=3, column=0, padx=10, pady=5, sticky="e")

        self.case_sensitive_var = tk.BooleanVar()
        self.case_sensitive_check = tk.Checkbutton(root, text="Case Sensitive", font=label_font, variable=self.case_sensitive_var, fg="#e06c75", bg="#282c34", selectcolor="#282c34")
        self.case_sensitive_check.grid(row=3, column=1, padx=10, pady=5, sticky="w")

        # Search Button
        self.search_button = tk.Button(root, text="Search", font=button_font, command=self.search, fg="#ffffff", bg="#98c379")
        self.search_button.grid(row=4, column=0, columnspan=2, padx=10, pady=15)

        # Results Text Box
        self.result_text = tk.Text(root, width=60, height=20, font=("Courier", 10), bg="#1e2127", fg="#abb2bf", borderwidth=0, highlightthickness=1, highlightbackground="#61afef")
        self.result_text.grid(row=5, column=0, columnspan=2, padx=10, pady=10)

    def load_files(self):
        # Open file dialog to select multiple text files
        filepaths = filedialog.askopenfilenames(title="Select Text Files", filetypes=(("Text files", "*.txt"), ("All files", "*.*")))
        if not filepaths:
            messagebox.showwarning("No Files Selected", "Please select at least one text file.")
            return

        # Clear previously loaded contents
        self.file_contents.clear()

        # Load each file and store its content
        for filepath in filepaths:
            try:
                with open(filepath, 'r', encoding='utf-8') as file:
                    content = file.read()
                    self.file_contents[filepath] = content
            except Exception as e:
                messagebox.showerror("File Load Error", f"Could not load {filepath}. Error: {e}")
                continue

        # Show a success message after loading files
        if self.file_contents:
            messagebox.showinfo("Files Loaded", f"Loaded {len(self.file_contents)} files successfully.")

    def search(self):
        if not self.file_contents:
            messagebox.showwarning("No Files Loaded", "Please load text files before searching.")
            return

        search_term = self.search_term_entry.get()
        if not search_term:
            messagebox.showerror("Error", "Please enter a search term.")
            return

        whole_word = self.whole_word_var.get()
        case_sensitive = self.case_sensitive_var.get()

        start_time = time.time()
        brute_results = {file: brute_force_search(content, search_term, case_sensitive, whole_word)
                         for file, content in self.file_contents.items()}
        brute_time = time.time() - start_time

        start_time = time.time()
        kmp_results = {file: kmp_search(content, search_term, case_sensitive, whole_word)
                       for file, content in self.file_contents.items()}
        kmp_time = time.time() - start_time

        # Display results in the result text box
        self.result_text.delete("1.0", tk.END)
        self.result_text.insert(tk.END, f"Brute Force Search Time: {brute_time:.5f} seconds\n", "header")
        self.result_text.insert(tk.END, f"KMP Search Time: {kmp_time:.5f} seconds\n\n", "header")
        self.result_text.insert(tk.END, "Results:\n\n", "header")

        for file, results in brute_results.items():
            self.result_text.insert(tk.END, f"{file}:\n", "filename")
            for row, col in results:
                self.result_text.insert(tk.END, f"Brute Force - Row {row}, Col {col}\n", "result")

        for file, results in kmp_results.items():
            self.result_text.insert(tk.END, f"{file}:\n", "filename")
            for row, col in results:
                self.result_text.insert(tk.END, f"KMP - Row {row}, Col {col}\n", "result")

        # Tag configurations for styling in the text box
        self.result_text.tag_config("header", foreground="#e06c75", font=("Helvetica", 10, "bold"))
        self.result_text.tag_config("filename", foreground="#61afef", font=("Helvetica", 10, "bold"))
        self.result_text.tag_config("result", foreground="#abb2bf", font=("Courier", 10))

if __name__ == "__main__":
    root = tk.Tk()
    app = WordSearchApp(root)
    root.mainloop()
